# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,-20.93,99,58,4.03,RU,1700168727
1,1,port-aux-francais,-49.3500,70.2167,5.07,93,100,13.12,TF,1700168682
2,2,waitangi,-43.9535,-176.5597,16.01,89,100,3.13,NZ,1700168686
3,3,stanley,54.8680,-1.6985,5.85,96,40,1.03,GB,1700168709
4,4,belyy yar,53.6039,91.3903,-6.06,100,100,0.00,RU,1700168728


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1,
    color="City",
    alpha= 0.5,
    tiles=True
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,30,laguna,38.4210,-121.4238,22.32,64,0,4.12,US,1700168710
33,33,carnarvon,-24.8667,113.6333,21.10,86,0,1.25,AU,1700168711
97,97,maun,-19.9833,23.4167,22.98,78,0,2.06,BW,1700168745
200,200,santa cruz de tenerife,28.4682,-16.2546,24.78,39,0,3.60,ES,1700168764
424,424,mahajanga,-15.7167,46.3167,26.93,72,0,2.18,MG,1700168811
425,425,freetown,8.4840,-13.2299,26.92,85,0,2.72,SL,1700168811
433,433,rengo,-34.4167,-70.8667,25.18,36,0,3.47,CL,1700168742
453,453,antonio enes,-16.2325,39.9086,24.16,72,0,4.08,MZ,1700168816
477,477,fortuna,40.5982,-124.1573,22.74,53,0,1.79,US,1700168819
487,487,tindouf,27.6711,-8.1474,22.13,21,0,3.51,DZ,1700168822


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
30,laguna,US,38.4210,-121.4238,64,
33,carnarvon,AU,-24.8667,113.6333,86,
97,maun,BW,-19.9833,23.4167,78,
200,santa cruz de tenerife,ES,28.4682,-16.2546,39,
424,mahajanga,MG,-15.7167,46.3167,72,
425,freetown,SL,8.4840,-13.2299,85,
433,rengo,CL,-34.4167,-70.8667,36,
453,antonio enes,MZ,-16.2325,39.9086,72,
477,fortuna,US,40.5982,-124.1573,53,
487,tindouf,DZ,27.6711,-8.1474,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
laguna - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
maun - nearest hotel: No hotel found
santa cruz de tenerife - nearest hotel: No hotel found
mahajanga - nearest hotel: No hotel found
freetown - nearest hotel: No hotel found
rengo - nearest hotel: No hotel found
antonio enes - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
30,laguna,US,38.4210,-121.4238,64,No hotel found
33,carnarvon,AU,-24.8667,113.6333,86,No hotel found
97,maun,BW,-19.9833,23.4167,78,No hotel found
200,santa cruz de tenerife,ES,28.4682,-16.2546,39,No hotel found
424,mahajanga,MG,-15.7167,46.3167,72,No hotel found
425,freetown,SL,8.4840,-13.2299,85,No hotel found
433,rengo,CL,-34.4167,-70.8667,36,No hotel found
453,antonio enes,MZ,-16.2325,39.9086,72,No hotel found
477,fortuna,US,40.5982,-124.1573,53,No hotel found
487,tindouf,DZ,27.6711,-8.1474,21,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    size="Humidity", 
    scale=1,
    color="City",
    alpha= 0.5,
    tiles=True,
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)